In [108]:
from collections import OrderedDict
import pandas as pd

def write_roman(num: int):
    roman = OrderedDict()
    roman[1000] = "M"
    roman[900] = "CM"
    roman[500] = "D"
    roman[400] = "CD"
    roman[100] = "C"
    roman[90] = "XC"
    roman[50] = "L"
    roman[40] = "XL"
    roman[10] = "X"
    roman[9] = "IX"
    roman[5] = "V"
    roman[4] = "IV"
    roman[1] = "I"

    def roman_num(num: int):
        for r in roman.keys():
            x, y = divmod(num, r)
            yield roman[r] * x
            num -= (r * x)
            if num <= 0:
                break

    return "".join([a for a in roman_num(num)])


def apply_replacements(text, replacements):
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

In [109]:
observatoriologistico_df = pd.read_csv('sources/observatoriologistico.csv', dtype=str)

observatoriologistico_df

,Código Región,Nombre Región,Abreviatura Región,Código Provincia,Nombre Provincia,Código Comuna 2018,Nombre Comuna
0,01,Tarapacá,TPCA,011,Iquique,01101,Iquique
1,01,Tarapacá,TPCA,011,Iquique,01107,Alto Hospicio
2,01,Tarapacá,TPCA,014,Tamarugal,01401,Pozo Almonte
3,01,Tarapacá,TPCA,014,Tamarugal,01402,Camiña
4,01,Tarapacá,TPCA,014,Tamarugal,01403,Colchane
...,...,...,...,...,...,...,...
341,16,Ñuble,NUBLE,163,Punilla,16301,San Carlos
342,16,Ñuble,NUBLE,163,Punilla,16302,Coihueco
343,16,Ñuble,NUBLE,163,Punilla,16303,Ñiquén
344,16,Ñuble,NUBLE,163,Punilla,16304,San Fabián


In [110]:
replacements = {
    'Aisén del General Carlos Ibañez del Campo': 'Aysén del General Carlos Ibáñez del Campo',
    'Magallanes': 'Magallanes y de la Antártica Chilena',
    'Región Metropolitana de Santiago': 'Metropolitana de Santiago'
}

iso_df = pd.read_csv('sources/iso.csv')
iso_df['corrected_name'] = iso_df['Subdivision name'].apply(lambda x: apply_replacements(x, replacements))
iso_df

,Subdivision category,3166-2 code,Subdivision name,Local variant,Language code,Romanization system,Parent subdivision,corrected_name
0,region,CL-AI*,Aisén del General Carlos Ibañez del Campo,"Aysén, Aisén",es,NaN,NaN,Aysén del General Carlos Ibáñez del Campo
1,region,CL-AN*,Antofagasta,NaN,es,NaN,NaN,Antofagasta
2,region,CL-AP*,Arica y Parinacota,NaN,es,NaN,NaN,Arica y Parinacota
3,region,CL-AT*,Atacama,NaN,es,NaN,NaN,Atacama
4,region,CL-BI*,Biobío,NaN,es,NaN,NaN,Biobío
5,region,CL-CO*,Coquimbo,NaN,es,NaN,NaN,Coquimbo
6,region,CL-AR*,La Araucanía,NaN,es,NaN,NaN,La Araucanía
7,region,CL-LI*,Libertador General Bernardo O'Higgins,O'Higgins,es,NaN,NaN,Libertador General Bernardo O'Higgins
8,region,CL-LL*,Los Lagos,NaN,es,NaN,NaN,Los Lagos
9,region,CL-LR*,Los Ríos,NaN,es,NaN,NaN,Los Ríos


In [111]:
def vectorized_write_roman(numbers):
    # Debe retornar una lista o Serie de números romanos
    return [write_roman(num) for num in numbers]

# Preprocesar iso_df para hacer la correspondencia más directa y limpiar el código ISO
iso_df['clean_iso_code'] = iso_df['3166-2 code'].str.replace('*', '')

# Crear una columna en observatoriologistico_df con la representación romana de 'Código Región'
observatoriologistico_df['prefix'] = vectorized_write_roman(observatoriologistico_df['Código Región'].astype(int))

# Unir los dataframes en los nombres corregidos y necesarios
merged_df = observatoriologistico_df.merge(iso_df[['corrected_name', 'clean_iso_code']], left_on='Nombre Región', right_on='corrected_name', how='left')

# Seleccionar y renombrar las columnas necesarias para el DataFrame final
regions_df = merged_df[['Código Región', 'Nombre Región', 'Abreviatura Región', 'prefix', 'clean_iso_code']]
regions_df.columns = ['id', 'name', 'abbreviation', 'prefix', 'iso_code']
regions_df = regions_df.drop_duplicates(subset=['id'], keep='last')

# Guardar a CSV y mostrar el DataFrame
regions_df.to_csv('region_data.csv', index=False)
regions_df


,id,name,abbreviation,prefix,iso_code
6,01,Tarapacá,TPCA,I,CL-TA
15,02,Antofagasta,ANTOF,II,CL-AN
24,03,Atacama,ATCMA,III,CL-AT
39,04,Coquimbo,COQ,IV,CL-CO
77,05,Valparaíso,VALPO,V,CL-VS
110,06,Libertador General Bernardo O'Higgins,LGBO,VI,CL-LI
140,07,Maule,MAULE,VII,CL-ML
173,08,Biobío,BBIO,VIII,CL-BI
205,09,La Araucanía,ARAUC,IX,CL-AR
235,10,Los Lagos,LAGOS,X,CL-LL


In [112]:
provinces_df = observatoriologistico_df[['Código Provincia', 'Nombre Provincia', 'Código Región']].copy()
provinces_df.columns = ['id', 'name', 'region_id']

provinces_df = provinces_df.drop_duplicates(subset=['id'], keep='last')

provinces_df.to_csv('province_data.csv', index=False)
provinces_df

,id,name,region_id
1,011,Iquique,01
6,014,Tamarugal,01
10,021,Antofagasta,02
13,022,El Loa,02
15,023,Tocopilla,02
18,031,Copiapó,03
20,032,Chañaral,03
24,033,Huasco,03
30,041,Elqui,04
34,042,Choapa,04


In [113]:
communes_df = observatoriologistico_df[['Código Comuna 2018', 'Nombre Comuna', 'Código Provincia']].copy()
communes_df.columns = ['id', 'name', 'province_id']

communes_df.to_csv('commune_data.csv', index=False)
communes_df

,id,name,province_id
0,01101,Iquique,011
1,01107,Alto Hospicio,011
2,01401,Pozo Almonte,014
3,01402,Camiña,014
4,01403,Colchane,014
...,...,...,...
341,16301,San Carlos,163
342,16302,Coihueco,163
343,16303,Ñiquén,163
344,16304,San Fabián,163


In [114]:
import json

# Cargar tus DataFrames
# Suponiendo que los DataFrames ya están limpios y contienen las columnas necesarias:
# regions_df con columnas ['id', 'name', 'iso_code']
# provinces_df con columnas ['id', 'name', 'region_id']
# communes_df con columnas ['id', 'name', 'province_id']

# Primero, construimos la jerarquía desde el nivel más bajo, las comunas
# Agrupar comunas por provincia_id y construir el diccionario de comunas
communes_grouped = communes_df.groupby('province_id')
provinces_df['communes'] = provinces_df['id'].apply(lambda x: {row['id']: {'id': row['id'], 'name': row['name']} for _, row in communes_grouped.get_group(x).iterrows() if x in communes_grouped.groups})

# Agrupar provincias por region_id y construir el diccionario de provincias
provinces_grouped = provinces_df.groupby('region_id')
regions_df['provinces'] = regions_df['id'].apply(lambda x: {row['id']: {'id': row['id'], 'name': row['name'], 'communes': row['communes']} for _, row in provinces_grouped.get_group(x).iterrows() if x in provinces_grouped.groups})

# Construir el diccionario final para el país, asumiendo que el país es Chile
cl_country = {
    'country': 'Chile',
    'regions': {row['id']: {'id': row['id'], 'name': row['name'], 'iso_code': row['iso_code'], 'provinces': row['provinces']} for _, row in regions_df.iterrows()}
}

# Convertir el diccionario a JSON
json_data = json.dumps(cl_country, ensure_ascii=False, indent=2)

# Guardar JSON en un archivo
with open('data.json', 'w', encoding='utf-8') as f:
    f.write(json_data)

# Imprimir el JSON para verificar
print(json_data)


{
  "country": "Chile",
  "regions": {
    "01": {
      "id": "01",
      "name": "Tarapacá",
      "iso_code": "CL-TA",
      "provinces": {
        "011": {
          "id": "011",
          "name": "Iquique",
          "communes": {
            "01101": {
              "id": "01101",
              "name": "Iquique"
            },
            "01107": {
              "id": "01107",
              "name": "Alto Hospicio"
            }
          }
        },
        "014": {
          "id": "014",
          "name": "Tamarugal",
          "communes": {
            "01401": {
              "id": "01401",
              "name": "Pozo Almonte"
            },
            "01402": {
              "id": "01402",
              "name": "Camiña"
            },
            "01403": {
              "id": "01403",
              "name": "Colchane"
            },
            "01404": {
              "id": "01404",
              "name": "Huara"
            },
            "01405": {
              "id": "